# Quality evaluation

## BUSCO
- Mandatory arguments
    - `busco -i [SEQUENCE_FILE] -l [LINEAGE] -o [OUTPUT_NAME] -m [MODE] [OTHER OPTIONS]`
- -i or --in defines the input file to analyse which is either a nucleotide fasta file or a protein fasta file, depending on the BUSCO mode. As of v5.1.0 the input argument can now also be a directory containing fasta files to run in batch mode.
- -o or --out defines the folder that will contain all results, logs, and intermediate data
- -m or --mode sets the assessment MODE: genome, proteins, transcriptome
- -l or --lineage_dataset
    - It can be a dataset name, i.e. bacteria_odb10, or a path i.e. ./bacteria_odb10 or /home/user/bacteria_odb10. In the former case, which is the recommended usage, BUSCO will automatically download and version the corresponding dataset. In the latter case, the dataset found in the given path will be used. Lineage can be ignored if running automated lineage selection

In [89]:
import re 
import pandas as pd
spades = pd.read_csv('results/assembly/4-assess/busco/spades/multiqc/multiqc_data/multiqc_busco.txt', sep='\t').drop('lineage_dataset', axis='columns')
spades = spades.drop(['complete_single_copy', 'missing', 'total', 'complete_duplicated', 'fragmented'], axis='columns')
megahit = pd.read_csv('results/assembly/4-assess/busco/megahit/multiqc/multiqc_data/multiqc_busco.txt', sep='\t').drop('lineage_dataset', axis='columns')
megahit = megahit.drop(['complete_single_copy', 'missing', 'total', 'complete_duplicated', 'fragmented'], axis='columns')
for df in [spades, megahit]:
    extracted = df['Sample'].str.extract(r'.*(S.*)_(.*)')
    df['Sample'] = extracted[0]
    df['Assembler'] = extracted[1]
pd.merge(spades, megahit, on='Sample')

,Sample,complete_x,Assembler_x,complete_y,Assembler_y
0,S01,1297.0,spades,1251.0,megahit
1,S02,3423.0,spades,3352.0,megahit
2,S03,3626.0,spades,3641.0,megahit
3,S04,3595.0,spades,3621.0,megahit
4,S05,3025.0,spades,3006.0,megahit
5,S06,3659.0,spades,3655.0,megahit
6,S07,3647.0,spades,3668.0,megahit
7,S08,3674.0,spades,3673.0,megahit
8,S09,3318.0,spades,3490.0,megahit
9,S10,3669.0,spades,3661.0,megahit


# MAKER
Annotation by maker proceeds in several rounds, each one building upon the last. 
Based on [this](https://github.com/Joseph7e/MAKER-genome-annotations-tutorial) tutorial

### Constructing repeat database

Before annotation, it is important to prepare a repeat database to prevent gene predictors from including known repeats (e.g. LTRs, tandem repeats) in their analyses. Since the standard resource for this, RepBase, is now behind a paywall, we need to prepare our own species-specific database. This will be carried out with the software RepeatModeler and the *Sporothrix schenckii* reference genome

In [ ]:
BuildDatabase -name <name> reference-genome.fasta
RepeatModeler -database <species blast database> -LTRStruct 

Since repeat modeler having identified ... repeat elements, you'll have to use additional tools to increase the size of the repeat element library.
- EDTA 
    - will be run with the`--curatedlib` flag set to the (identified) output of RepeatModeler.
- MITE hunter
 - This uses the genome only
 - 
- TransposonPSI
- Downloaded a curated transposable element sequences from [TREP](https://trep-db.uzh.ch/index.php) database

In [ ]:
# EDTA
perl EDTA.pl --genome S_schenckii_genome.fasta --curatedlib <repeatmodeler file> 

### Round 1
In the first round, set maker to infer gene predictions from all ESTs and from protein alignments. 
- Use the `gff3` file output to train augustus and snap

In [ ]:
est2genome=1 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=1 #infer predictions from protein homology, 1 = yes, 0 = no

### Round 2

After the second round, Galaxy recommends to train the predictors and second time using the output of the maker second run. Once done, just rerun Maker a third time using the same params as the second round. 
- Since there will be multiple gff files though for each contig, we'll instead merge the files together, then extract the est, protein and repeat element annotations into separate files for use instead

In [ ]:
gff3_merge -n -s -d <sample>_master_datastore_index.log > <sample>.all.maker.noseq.gff
# transcript alignments
awk '{ if ($2 == "est2genome") print $0 }' ${sample}.all.maker.noseq.gff > all.maker.est2genome.gff
# protein alignments
awk '{ if ($2 == "protein2genome") print $0 }' .all.maker.noseq.gff > .all.maker.protein2genome.gff
# repeat alignments
awk '{ if ($2 ~ "repeat") print $0 }' .all.maker.noseq.gff > .all.maker.repeats.gff

In [ ]:
#-----Re-annotation Using MAKER Derived GFF3
maker_gff= #MAKER derived GFF3 file, output of the previous maker run
est_pass=1 #use ESTs in maker_gff: 1 = yes, 0 = no
altest_pass=1 #use alternate organism ESTs in maker_gff: 1 = yes, 0 = no 
protein_pass=1 #use protein alignments in maker_gff: 1 = yes, 0 = no
rm_pass=1 #use repeats in maker_gff: 1 = yes, 0 = no
model_pass=0 #use gene models in maker_gff: 1 = yes, 0 = no
pred_pass=0 #use ab-initio predictions in maker_gff: 1 = yes, 0 = no
other_pass=0 #passthrough anyything else in maker_gff: 1 = yes, 0 = no

# Set both the snap and augustus lines to the output of the training
#-----Gene Prediction
snaphmm= #SNAP HMM file 
gmhmm= #GeneMark HMM file 
augustus_species= #Augustus gene prediction species model
fgenesh_par_file= #FGENESH parameter file
pred_gff= #ab-initio predictions from an external GFF3 file
model_gff= #annotated gene models from an external GFF3 file (annotation pass-through)
run_evm=0 #run EvidenceModeler, 1 = yes, 0 = no
# Will also disable these
est2genome=0 #infer gene predictions directly from ESTs, 1 = yes, 0 = no
protein2genome=0 #infer predictions from protein homology, 1 = yes, 0 = no
#
trna=0 #find tRNAs with tRNAscan, 1 = yes, 0 = no
snoscan_rrna= #rRNA file to have Snoscan find snoRNAs
snoscan_meth= #-O-methylation site fileto have Snoscan find snoRNAs
unmask=0 #also run ab-initio prediction programs on unmasked sequence, 1 = yes, 0 = no
allow_overlap= #allowed gene overlap fraction (value from 0 to 1, blank for default)


## Config files
- Run `maker -CTL` to generate all the config files
- Set `model_org=` to empty until we find a way to get RepBase

## Output file info

### *Ab initio* gene prediction
To train the predictors, pass it the original genome file, as well as the fasta output from the first round of maker

#### SNAP
***The following describes steps from maker output***
- Preparation
	- Convert the GFF3 gene models into ZFF format
	- Make a directory, move into it, and run `gff3_merge -d` using the datastore index log as an argument to merge the gff files into one
	- Then run `maker2zff`

In [ ]:
mkdir <snap directory>
cd <snap directory>
gff3_merge -d <master_datastore_index.log>
maker2zff <the merged gff file> # Creates .ann and .dna files

- Training

In [ ]:
fathom -categorize <intergenic sequence limit> <genome>.ann <genome>.dna
fathom <>.ann <>.dna -gene-stats > gene-stats.log 2>&1 # Get stats
fathom <>.ann <>.dna -validate > validate.log 2>&1 # Validate
fathom -export 1000 -plus uni.ann uni.dna # plus converts the sequence to plus strand 
forge export.ann export.dna
hmm-assembler.pl <name> <file directory from forge export, usually current dir>  > <training parameters file name>


Once the training parameter file has been created, update the `maker_opts.ctl` file, setting the following 

In [ ]:
snaphmm=<training file name>
est2genome=0
protein2genome=0

### Augustus
Protocol from https://biohpc.cornell.edu/doc/annotation_2019_exercises1_v2.pdf
- To start off, create a merged gff file the same way you did for snap
- Next, filter the file so it only contains maker annotations  
- Convert the maker gff and fasta file into a genbank formatted file, keeping 2000 np upstream and downstream of each gene for model training
- Create a new augustus species, then perform initial training using the genbank file 
- You can check the number of genes in the training set using `grep -c LOCUS <gb file>`
- **Model training**
    - Create a new augustus species, then train it on the gb file as that species. It will be stored in the default augustus/config directory
    - Perform the initial training
    - Create a smaller test set (by splitting the gb file) for evaluating the model before and after obptimization
        - Use the first model to predict genes in the test set 
    - **Model optimization (very time consuming)**
        - To speed things up, you can train the model with a smaller test set 
        - This test set will be splitted into 24 kfolds for optimization (the kfold can be set up to 48, with processed with one cpu core per kfold. Kfold must be same number as as cpus). 
        - The training, prediction and evaluation will be performed on each bucket in parallel (training on hh.gb.train+each bucket, then comparing each bucket with the union of the rest). 
        - By default, 5 rounds of optimization. As optimization for large genome could take days, I changed it to 3 here

In [ ]:
# Option 2
gff3_merge -d <datastore_index.log>
awk '{if ($2=="maker") print }' <all.gff> > filtered_file.gff 
gff2gbSmallDNA.pl <gff file> <assembly fasta file> 2000 <genbank filename>.gb
new_species.pl --species=<your_species>

    # Training
etraining --species=<your_species> <genbank file>
randomSplit.pl <genbank file> <number of genes to split> # Yields a file called "<gb file name>.test" 
mv <filename>.gb.test <filename>.gb.evaluation # Rename for evaluation
augustus --species=<your_species> .gb.evaluation >& first_evaluation.out # First model testing
grep -A 22 Evaluation first_evaluation.out # Check results

# Optimize
randomSplit.pl <genbank file> 1000
optimize_augustus.pl --species=<your species> --kfold=24 --cpus=24 --rounds=3 --onlytrain=<filename>.gb.train <filename>.gb.test >& log

# Train after optimization
etraining --species=<your_species> <genbank file>

# Testing the optimized model
augustus --species=<your_species> .gb.evaluation >& second_evaluation.out # Second model testing

#### Alternative: Training Augustus with BUSCO
First slice of the ends of mRNA annotations

### GenemarksES
All you need for this is the geome assembly file

In [ ]:
gmes_petap.pl -ES -sequence <assembly.fasta>  -fungus
# The target file is gmhmm.mod (it produces a bunch of files in the current directory)

# TODO
Look into repeat library construction
- https://weatherby.genetics.utah.edu/MAKER/wiki/index.php/Repeat_Library_Construction-Basic
- https://weatherby.genetics.utah.edu/MAKER/wiki/index.php/Repeat_Library_Construction-Advanced

# Ref sequences

## Reference genomes
Used in construction of Repeat library and for training Augustus
- NCBI RefSeq assembly GCF_000961545.1

- Additional repeat library construction


## Protein evidence for maker
- Search for "Sporothrix" on UniProtKB, Mon 15 May, 2023, Release 2023_02 

## Transcriptome 
Initial database: all Sporothrix sequences from the NCBI's Nucleotide database, 
- NCBI nucleotide database search query: "txid29907[Organism:exp] AND biomol_mrna[PROP]" Mon 15 May, 2023 
- A transcriptome assembled using rnaspades SRA accession number SRX6367452

# References
https://darencard.net/blog/2017-05-16-maker-genome-annotation/